# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [207]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor, RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [ ]:
spaceship = spaceship.dropna()
spaceship.reset_index(drop=True, inplace=True)

In [ ]:
spaceship['Cabin'] = spaceship['Cabin'].apply(lambda x: x.split('/')[0] if isinstance(x, str) else np.nan)
spaceship = spaceship.drop(columns=['Name'])
spaceship = spaceship.drop(columns=['PassengerId'])

**Perform Train Test Split**

In [ ]:
spaceship = pd.get_dummies(spaceship, columns=['HomePlanet','CryoSleep','Cabin','Destination','VIP'])

In [ ]:
spaceship_numerical = spaceship.select_dtypes(include=['number'])

In [ ]:
normalizer = MinMaxScaler()

In [ ]:
spaceship_numerical_norm = normalizer.fit_transform(spaceship_numerical)

In [ ]:
spaceship_numerical_norm_df = pd.DataFrame(spaceship_numerical_norm, columns=spaceship_numerical.columns)

In [ ]:
numerical_cols = spaceship.select_dtypes(include=['number']).columns
spaceship = spaceship.drop(columns=numerical_cols)

In [ ]:
spaceship_combined = pd.concat([spaceship, spaceship_numerical_norm_df], axis=1)

In [ ]:
boolean_cols = spaceship_combined.select_dtypes(include=['bool']).columns
spaceship_combined[boolean_cols] = spaceship_combined[boolean_cols].astype(int)

In [ ]:
spaceship_combined.info()

In [ ]:
# X-y split; features = X, target = y
features = spaceship_combined.drop(columns = ["Transported"])
target = spaceship_combined["Transported"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=1)

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Decision Trees

In [173]:
tree = DecisionTreeClassifier(max_depth=10)

In [175]:
tree.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=10)

In [177]:
y_pred_test_dt = tree.predict(X_test)
print(f"MAE, {mean_absolute_error(y_pred_test_dt, y_test): .2f}")
print(f"MSE, {mean_squared_error(y_pred_test_dt, y_test): .2f}")
print(f"RMSE, {root_mean_squared_error(y_pred_test_dt, y_test): .2f}")
print(f"R2 score, {tree.score(X_test, y_test): .2f}")

MAE,  0.23
MSE,  0.23
RMSE,  0.48
R2 score,  0.77


- Bagging and Pasting

In [ ]:
bagging_reg = BaggingClassifier(DecisionTreeClassifier(max_depth=20),
                               n_estimators=100, # number of models to use
                               max_samples = 1000)

In [ ]:
bagging_reg.fit(X_train, y_train)

In [185]:
y_pred_test_bag = bagging_reg.predict(X_test)

print(f"MAE {mean_absolute_error(y_pred_test_bag, y_test): .2f}")
print(f"RMSE {root_mean_squared_error(y_pred_test_bag, y_test): .2f}")
print(f"MSE {mean_squared_error(y_pred_test_bag, y_test): .2f}")
print(f"R2 score {bagging_reg.score(X_test, y_test): .2f}")

MAE  0.21
RMSE  0.46
MSE  0.21
R2 score  0.79


In [183]:
accuracy = accuracy_score(y_test, y_pred_test_bag)
accuracy

0.791981845688351

In [ ]:
pasting_reg = BaggingClassifier(DecisionTreeClassifier(max_depth=20), n_estimators=100,max_samples=1000,bootstrap=False) # For Pasting, set bootstrap to False

In [ ]:
pasting_reg.fit(X_train, y_train)

In [ ]:
y_pred_test_past = pasting_reg.predict(X_test)

print(f"MAE {mean_absolute_error(y_pred_test_past, y_test): .2f}")
print(f"RMSE {root_mean_squared_error(y_pred_test_past, y_test): .2f}")
print(f"MSE {mean_squared_error(y_pred_test_past, y_test): .2f}")
print(f"R2 score {pasting_reg.score(X_test, y_test): .2f}")

- Random Forests

In [191]:
forest = RandomForestClassifier(n_estimators=100,
                             max_depth=20)

In [193]:
forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=20)

In [195]:
y_pred_test_rf = forest.predict(X_test)
print(f"MAE {mean_absolute_error(y_pred_test_rf, y_test): .2f}")
print(f"MSE, {mean_squared_error(y_pred_test_rf, y_test): .2f}")
print(f"RMSE, {root_mean_squared_error(y_pred_test_rf, y_test): .2f}")
print(f"R2 score, {forest.score(X_test, y_test): .2f}")

MAE  0.22
MSE,  0.22
RMSE,  0.46
R2 score,  0.78


- Gradient Boosting

In [201]:
gb_reg = GradientBoostingClassifier(max_depth=20,
                                   n_estimators=100)

In [203]:
gb_reg.fit(X_train, y_train)

GradientBoostingClassifier(max_depth=20)

In [204]:
y_pred_test_gb = gb_reg.predict(X_test)

print(f"MAE, {mean_absolute_error(y_pred_test_gb, y_test): .2f}")
print(f"MSE, {mean_squared_error(y_pred_test_gb, y_test): .2f}")
print(f"RMSE, {root_mean_squared_error(y_pred_test_gb, y_test): .2f}")
print(f"R2 score, {gb_reg.score(X_test, y_test): .2f}")

MAE,  0.22
MSE,  0.22
RMSE,  0.47
R2 score,  0.78


- Adaptive Boosting

In [213]:
ada_reg = AdaBoostClassifier(n_estimators=100)

In [215]:
ada_reg.fit(X_train, y_train)

C:\Users\faval\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(n_estimators=100)

In [217]:
y_pred_test_ada = ada_reg.predict(X_test)

print(f"MAE, {mean_absolute_error(y_pred_test_ada, y_test): .2f}")
print(f"MSE, {mean_squared_error(y_pred_test_ada, y_test): .2f}")
print(f"RMSE, {root_mean_squared_error(y_pred_test_ada, y_test): .2f}")
print(f"R2 score, {ada_reg.score(X_test, y_test): .2f}")

MAE,  0.20
MSE,  0.20
RMSE,  0.45
R2 score,  0.80


Which model is the best and why?

Adaptive Boosting because predict 80% accuracy and has lowest MAE only 0.20